# Changer la répartition des lexèmes dans les classes flexionnelles et les genres

La redistribution coupe en chunks de même taille pour toutes les combinaisons disponibles pour la catégorie.

- adjectifs : un niveau CF
- verbes :
    - le niveau VI/VT/VD n'est pas touché
    - on redistribue sur un niveau CF
- noms : 
    - un niveau CF 
    - un niveau de genre
- autres : pas de modifications

In [238]:
# -*- coding: utf8 -*-
#########################IMPORTS############################################
from os.path import expanduser
import codecs, optparse
import re
import random as rd
import pandas as pd
import itertools as it
import sys,os,time
import string
import yaml, YamlDuplicates
import ParFuMor as PFM
from ParFuMor import *
import pickle

def ding():
    os.system('afplay /System/Library/Sounds/Submarine.aiff')


In [239]:
%store -r numerosKalaba
numeroKalaba="20-K%d"%numerosKalaba[0]
print numeroKalaba

20-K1


In [240]:
home = expanduser("~")
serie=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/%s/"%numeroKalaba

with open(serie+"Stems.yaml", 'r') as stream:
    stems=yaml.safe_load(stream)


In [241]:
def getLexemes(stemStructure):
    formes={}
    for element in stemStructure:
        if type(stemStructure[element]) is list:
#             formesCle=", ".join()
            formes[element]=stemStructure[element]
        else:
            formes.update(getLexemes(stemStructure[element]))
    return formes

In [242]:
def recursive_items(dictionary):
    result={}
    for key, value in dictionary.items():
        if type(value) is dict:
            result[key]=recursive_items(value)
        else:
            result=[]
    return result
        

In [243]:
lexCF=getLeaves(stems["NOM"])
lexCF

{'Dafib': ['lit', 'lits'],
 'Dazep': [u'for\xeat', u'for\xeats'],
 'DomeZ': ['fruit', 'fruits'],
 'DotoSut': ['lance', 'lances'],
 'Duvop': ['table', 'tables'],
 'Negegiz': ['main', 'mains'],
 'NoTeN': ['villageois',
  'villageoiS',
  'villagEOISe',
  'villagEOISes',
  'villagEOIS',
  'villagEOIs'],
 'NoTim': ['villageoise', 'villageoises'],
 'Sagobul': [u'col\xe8re', u'col\xe8res'],
 'TeniS': ['loup', 'loups', 'lOUP', 'lOUPs'],
 'TezuT': ['viande', 'viandes'],
 'TiledaS': ['souffrance', 'souffrances'],
 'TiniSum': ['louve', 'louves'],
 'Togoniw': ['coyote', 'coyotes'],
 'ZegoN': [u'gar\xe7on', u'gar\xe7ons'],
 'benov': ['lune', 'lunes'],
 'bibuS': ['ombre', 'ombres'],
 'bovos': ['visage', 'visages'],
 'divuS': ['lueur', 'lueurs'],
 'dozut': [u'infirmi\xe8re', u'infirmi\xe8res'],
 'duZen': ['enfant', 'enfants'],
 'duzunuT': ['fille', 'filles'],
 'fezil': ['coup', 'coups'],
 'figoT': ['gorge', 'gorges'],
 'foZiDos': ['arbre', 'arbres'],
 'fovub': [u'caf\xe9', u'caf\xe9s'],
 'gaZur': ['c

In [244]:
def cutList(a, n):
    k, m = divmod(len(a), n)
    return (a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))


In [245]:
newStems={}
for cat in stems:
    print cat
    cfCat=recursive_items(stems[cat])
    if type(cfCat) is dict:
        if cat=="ADJ":
            newStems["ADJ"]={}
            adjectifs=list(getLexemes(stems["ADJ"]).items())
            adjectifsCF=stems["ADJ"].keys()
#             print adjectifsCF
            for nCF,cfAdjectifs in enumerate(cutList(adjectifs,len(adjectifsCF))):
#                 print {rad:v for rad,v in cfAdjectifs}
                newStems["ADJ"][adjectifsCF[nCF]]={rad:v for rad,v in cfAdjectifs}
        elif cat=="VER":
            newStems["VER"]={}
            if "VI" in cfCat.keys():
                for trans in "VI VT VD".split(" "):
                    print trans
                    newStems["VER"][trans]={}
                    verbes=list(getLexemes(stems["VER"][trans]).items())
                    verbesCF=stems["VER"][trans].keys()
                    for nCF,cfVerbes in enumerate(cutList(verbes,len(verbesCF))):
    #                     print {rad:v for rad,v in cfVerbes}
                        newStems["VER"][trans][verbesCF[nCF]]={rad:v for rad,v in cfVerbes}
            else:
                verbes=list(getLexemes(stems["VER"]).items())
                verbesCF=stems["VER"].keys()
                for nCF,cfVerbes in enumerate(cutList(verbes,len(verbesCF))):
#                     print {rad:v for rad,v in cfVerbes}
                    newStems["VER"][verbesCF[nCF]]={rad:v for rad,v in cfVerbes}
                
        elif cat=="NOM":
            newStems["NOM"]={}
            nomsL1=stems["NOM"].keys()
            nbNomL1=len(nomsCF)
            noms=list(getLexemes(stems["NOM"]).items())
            if stems["NOM"][nomsL1[0]] is dict:
                nomsL2=stems["NOM"][nomsL1[0]]
                nbNomL2=len(nomsL2)
                nbChunks=nbNomL1*nbNomL2
                nomsChunks=list(cutList(noms,nbChunks))
                for nChunk,(l1,l2) in enumerate(it.product(nomsL1,nomsL2)):
                    if l1 not in newStems["NOM"]:
                        newStems["NOM"][l1]={}
#                     print l1,l2
#                     print {rad:formes for rad,formes in nomsChunks[nChunk]}
                    newStems["NOM"][l1][l2]={rad:formes for rad,formes in nomsChunks[nChunk]}
            else:
                nbChunks=nbNomL1
                nomsChunks=list(cutList(noms,nbChunks))
                for nChunk,l1 in enumerate(nomsL1):
                    if l1 not in newStems["NOM"]:
                        newStems["NOM"][l1]={}
#                     print l1
#                     print {rad:formes for rad,formes in nomsChunks[nChunk]}
                    newStems["NOM"][l1]={rad:formes for rad,formes in nomsChunks[nChunk]}
        else:
            newStems[cat]=stems[cat]
    else:
        newStems[cat]=stems[cat]
print newStems

ADJ
DET
VER
PREP
NOM
{'VER': {'V1': {'nivap': [u'r\xeavasser', u'r\xeavasse', u'r\xeavassent', u'r\xeavassait', u'r\xeavassaient'], 'vitural': ['rejoindre', 'rejoint', 'rejoignent', 'rejoignait', 'rejoignaient'], 'juNoleg': ['tomber', 'tombe', 'tombent', 'tombait', 'tombaient', 'tomba', u'tomb\xe8rent'], 'poSob': ['arriver', 'arrive', 'arrivent', 'arrivait', 'arrivaient', 'arriva', u'arriv\xe8rent'], 'Nivumil': ['courir', 'court', 'courent', 'courait', 'couraient', 'courut', 'coururent'], 'keZuNup': ['envahir', 'envahit', 'envahissent', 'envahissait', 'envahissaient'], 'deven': ['fuir', 'fuit', 'fuient', 'fuyait', 'fuyaient'], 'Tizok': ['boire', 'boit', 'boivent', 'buvait', 'buvaient'], 'marufil': ['voir', 'voit', 'voient', 'voyait', 'voyaient', 'vit', 'virent'], 'Neranuf': ['sauter', 'saute', 'sautent', 'sautait', 'sautaient'], 'mawes': [u'd\xe9vorer', u'd\xe9vore', u'd\xe9vorent', u'd\xe9vorait', u'd\xe9voraient'], 'DuZob': ['aller', 'va', 'vont', 'allait', 'allaient', 'alla', u'all\

In [246]:
with open(serie+'NewStems.yaml', 'w') as output:
    yaml.dump(newStems, output, default_flow_style=False,allow_unicode=True)

In [247]:
with open(serie+"NewStems.yaml", 'r') as input:
    yamlLines=input.readlines()

In [248]:
yamlText="".join(yamlLines)
yamlText=re.sub(r"\n\s*-\s*",", ",yamlText)
yamlText=re.sub(r":,\s*",": ",yamlText)
yamlText=re.sub(r": *([^\n]+)",": [\g<1>]",yamlText)
print yamlText

ADJ:
  Sogeges: [terrible, terribles]
  Sozup: [différent, différents, différente, différentes]
  baZowa: [bas, basse, basses]
  foder: [jaune, jaunes]
  gadin: [maigre, maigres]
  guNomaS: [gros, grosse, grosses]
  jemibo: [profond, profonds, profonde, profondes]
  joZogiN: [courageux, courageuse, courageuses]
  kavijo: [effrayé, effrayés, effrayée, effrayées]
  kizow: [noir, noire, noirs, noires]
  milape: [grand, grande, grands, grandes]
  nemiT: [blanc, blanche, blancs, blanches]
  poleS: [furieux, furieuse, furieuses]
  rumiT: [rouge, rouges]
  ruzet: [petit, petite, petits, petites]
  sejin: [quatre]
  vojuN: [trois]
  webinur: [suivant, suivants, suivante, suivantes]
  wibam: [blessé, blessés, blessée, blessées]
DET:
  ke: [IND, un, une, des, dE]
  la: [DEF, le, la, les, l]
  p: [DEM, ce, cet, cette, ces]
NOM:
  F:
    Duvop: [table, tables]
    NoTeN: [villageois, villageoiS, villagEOISe, villagEOISes, villagEOIS, villagEOIs]
    NoTim: [villageoise, villageoises]
    Sagobul: 

In [249]:
with open(serie+'NewStems.yaml', 'w') as output:
    output.write(yamlText)

In [250]:
ding()